Ensambles de modelos
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/machine-learning/blob/master/01-archivos-y-directorios.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/machine-learning/blob/master/01-archivos-y-directorios.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/machine-learning/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/machine-learning/blob/master/readme.md)

# Definición del problema real

Se desea construir un esquema de pronóstico que permita aprovechar la información capturada por muchos modelos diferentes construidos sobre los mismos datos.

# Definición del problema en términos de los datos

Si se tienen múltiples pronósticos obtenidos por diferentes modelos para los mismos datos, se mejor simplemente seleccionar el mejor mejor modelo y descartar la demás información, o se pueden aprovechar dichos pronósticos.

# Solución

Un ensamble es un tipo de modelo que permite la combinación de varios modelos de predicción para obtener un solo pronóstico basado en los pronósticos individuales de cada modelo. En la figura siguiente se presenta un esquema ilustrativo.

![alt text](images/combiner.jpg)



La clave de la operación de esta metodología se basa en la diversidad, la cual puede obtenerse de diferentes formas:

* Variando los datos de entrenamiento: se usa el mismo modelo en todos los casos, pero para cada uno de ellos se usa una muestra de entrenamiento obtenida por boostraping; así cada modelo tiene parámetros diferentes


* Variando la configuración del modelo: se usan exactamente los mismos datos de entrenamiento, pero sobre diferentes modelos obtenidos variando su configuración, por ejemplo mismo modelo pero con diferente configuración (entradas usadas, complejidad, etc). Inclusive se pueden utilizar distintos modelos.


* Una combinación de los dos anteriores.

El combinador es un mecanismo que obtiene un único pronóstico a partir de los pronósticos individuales de cada modelo. Para problemas de clasificación, la convinación se hace por votación. En problemas de regresión, mediante promedio simple, promedio combinado o, inclusive, regresión.

Diferentes metodologías se han desarrollado sobre este concepto.

### Bagging (Bootstrap aggregation)

En esta metodología la diversidad se obtiene al entrenar un mismo modelo sobre diferentes conjuntos de entrenamiento usando bootstrap. La combinación se hace por votación para problemas de clasificación y por promedio para problemas numéricos.

In [1]:
# install.packages("ipred")
library(ipred)


The downloaded binary packages are in
	/var/folders/yq/svn60mh123z6dzr3d4rjk_740000gn/T//Rtmp4KXWMk/downloaded_packages


In [5]:
# crea 25 arboles de decisión
credit <- read.csv("data/credit.csv")
credit$default <- factor(credit$default, labels=c("No", "Yes"))
mybag <- bagging(default ~ ., data = credit, nbagg = 25)
credit_pred <- predict(mybag, credit)
table(credit_pred, credit$default)

           
credit_pred  No Yes
        No  700   0
        Yes   0 300

In [6]:
# Para evaluar realmente la precisión del modelo
# se usa crossvalidation
library(caret)
set.seed(300)
ctrl <- trainControl(method = "cv", number = 10)
train(default ~ ., data = credit, method = "treebag", trControl = ctrl)
# note que tiene un desempeño muy similar al metodo C5.0

Loading required package: lattice
Loading required package: ggplot2


Bagged CART 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results:

  Accuracy  Kappa  
  0.746     0.36038


### AdaBoosting (Adaptive Boosting)

En este caso, los conjuntos de dato son diseñados especificamente para generar modelos complementarios. De forma simplificada, el algoritmo procede de la siguiente forma:

* Paso 1: Se construye un clasificador sobre todos los datos de entrenamiento.


* Paso 2: Se construye un nuevo conjunto de datos con los ejemplos mal clasificados (o una porción de ellos).


* Paso 3: se construye un nuevo clasificador con los datos obtenidos en el paso 2.


* Paso 4: Se retorna al Paso 2.


El proceso itera hasta que se alcanza una precisión requerida o el número máximo de clasificadores permitidos en el ensamble.

In [9]:
# install.packages("adabag")
library(adabag)
set.seed(300)
m_adaboost <- boosting(default ~ ., data = credit)
p_adaboost <- predict(m_adaboost, credit)
p_adaboost$confusion

Loading required package: rpart
Loading required package: foreach
Loading required package: doParallel
Loading required package: iterators
Loading required package: parallel

Attaching package: ‘adabag’

The following object is masked from ‘package:ipred’:

    bagging



               Observed Class
Predicted Class  No Yes
            No  700   0
            Yes   0 300

In [10]:
# verificación usando CV
set.seed(300)
adaboost_cv <- boosting.cv(default ~ ., data = credit)
adaboost_cv$confusion

i:  1 Sun Mar  4 09:58:05 2018 
i:  2 Sun Mar  4 09:58:39 2018 
i:  3 Sun Mar  4 09:59:13 2018 
i:  4 Sun Mar  4 09:59:47 2018 
i:  5 Sun Mar  4 10:00:21 2018 
i:  6 Sun Mar  4 10:00:56 2018 
i:  7 Sun Mar  4 10:01:31 2018 
i:  8 Sun Mar  4 10:02:07 2018 
i:  9 Sun Mar  4 10:02:43 2018 
i:  10 Sun Mar  4 10:03:18 2018 


               Observed Class
Predicted Class  No Yes
            No  594 153
            Yes 106 147

In [12]:
# install.packages("vcd")
library(vcd)
Kappa(adaboost_cv$confusion)

Loading required package: grid


            value     ASE     z Pr(>|z|)
Unweighted 0.3544 0.03237 10.95 6.68e-28
Weighted   0.3544 0.03237 10.95 6.68e-28

### Random forest

En este método se combinan el bagging con la selección aleatoria de caracteríticas para aumetar la diversidad. La salida del modelo se obtiene por votación. Cada ejemplo que no es considerado durante el entrenamiento es usado como parte del conjunto de prueba.

In [13]:
# install.packages("randomForest")
library(randomForest)
set.seed(300)
# por defecto crea un ensamble de 500 arboles
# que consideran la raiz cuadrada de la cantidad
# de atributos presentes en el conjunto de entrenamiento
rf <- randomForest(default ~ ., data = credit)
rf
# en la salida OOB se refiere a out-of-bag error rate

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin




Call:
 randomForest(formula = default ~ ., data = credit) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 4

        OOB estimate of  error rate: 22.7%
Confusion matrix:
     No Yes class.error
No  649  51  0.07285714
Yes 176 124  0.58666667

In [14]:
# A continuación se comparan los resultados de 
# random forest (rf) con C5.0 
library(caret)
# parametros de control
ctrl <- trainControl(method = "repeatedcv",
                     number = 10, 
                     repeats = 10)
# malla de parametros a considerar
grid_rf <- expand.grid(.mtry = c(2, 4, 8, 16))
set.seed(300)
m_rf <- train(default ~ ., 
              data = credit, 
              method = "rf",
              metric = "Kappa", 
              trControl = ctrl,
              tuneGrid = grid_rf)
m_rf 

In [ ]:
# comparación con un boosted tree
grid_c50 <- expand.grid(.model = "tree",
                        .trials = c(10, 20, 30, 40),
                        .winnow = "FALSE")
set.seed(300)
m_c50 <- train(default ~ ., 
               data = credit, 
               method = "C5.0",
               metric = "Kappa", 
               trControl = ctrl,
               tuneGrid = grid_c50)
m_c50

**Ejercicio.--** Cuál de los dos modelos anteriores fue mejor?